### 第一章 Node简介 1

一个Node进程只允许一个线程处理js。一个大型网络应用，可以由多个Node进程通过通信来协作构成，每个Node进程都构成这个应用中的一个节点，这是Node名字的本意。在分布式中，各个单点之间保持并行非阻塞异步，在这样的组织方式下，执行任务速度更快。

js作为一门图灵完备的语言，长久以来被禁锢在浏览器中，和其他语言一样，js的能力取决于宿主提供的支持有多少。浏览器的下层是操作系统，网卡，硬盘，显卡。Node内部基于libuv实现跨平台。libuv称为平台层，在Node程序中，在操作系统的上层。libuv处理事件循环/文件操作不同操作系统的实现

程序Node和chrome是对等物。他们实现的都是基于轮询事件的异步架构。轮询事件 === 循环查询事件\监听事件 === 在tick中依次查看各类观察者是否有事件，执行对应回调。

单线程的最大好处是不用像多线程编程那样处处在意“状态”同步，不用锁。没有线程上下文的切换。

客户端，发起请求端。服务端，响应请求端。Node可以做客户端也可以做服务端。

### 第二章 模块机制 

文件 === 模块 === 函数 === 对象。

每个模块被包装成函数，函数以exports,require,module,__filename,__dirname为输入。exports就是module.exports。通过module.paths读出所有可读的node_modules目录。该函数被runInThisContext调用。

Node导入模块，第一次导入时，会调用该模块并缓存返回对象，第二次导入时，Node直接返回该对象。用户模块返回对象缓存在Module._cache，以文件路径为key。库模块返回对象缓存在NativeModule._cache。

Node由commonjs和es规范。commonjs涵盖模块系统/二进制处理/Buffer/字符集编码/流/进程调用/文件系统fs/套接字/单元测试/包管理。commonjs规范让js具备开发大型应用比如服务器程序/客户端程序/命令行程序的能力。

引入模块不指定文件类型，Node会以不同文件类型(js/json/node/目录)查找该文件。其他后缀类型的文件以js方式处理。如果查找到目录，会查找该目录的package.json的main属性指定的文件，若没有，则以目录内的index.js/index.json/index.node文件作为该目录的入口文件。

库模块除了js模块，还存在使用c++的模块（如fs/buffer/os模块）。库模块中的c++源码，在Node被编译成二进制时，一起被编译。
库模块的js源码放在Node项目的lib目录，库模块的c++源码放在src中。在编译c++前，会把lib的js源码放入c++的字符串数组，借助js2c.py。引入js库模块时，从内存中取得js代码字符串，也是包装成函数后调用。执行完后缓存在NativeModule._cache对象中。与用户模块的区别是，用户模块从硬盘中取代码字符串。

c++用户模块先编译为node文件（node文件 === windows下的dll文件 === linux的so文件）。通过执行node-gyp configure和node-gyp build编译得到node文件。引入node文件，即引入c++用户模块，底层调用dlopen函数。

Node项目通过gyp编译。linux g++/gcc === window vc++

安装本地包 npm i <含package.json的文件夹>，安装远程包npm i url。package.json的scripts用来定义钩子函数，例如key为install，在npm install时自动执行

核心模块调内建模块，内建模块调内核API。node中利用C++拓展调用内核API

### 第三章 异步IO 1

io输入 === 把数据从io设备带到内存 === 把数据输入cpu。

异步关系 === 不存在相互约束的关系。同步关系 === 存在相互约束的关系。如果前一行代码阻塞后一行代码，两行代码就构成了同步的关系。阻塞，只是同步的一种方式。又比如把网络数据带到本地内存的操作完成前可以调用其他操作，io操作没有阻塞其他操作，那么他们就是异步关系。

从硬件结构上，计算机可以在同一时刻进行io操作和cpu计算。但由于历史原因，操作系统提供了阻塞io，让应用程序中的cpu计算需要等待io操作完成。

linux用文件表示io设备。应用程序需要使用文件符进行io操作。文件符是操作系统暴露给应用程序的操作对象。

调用非阻塞io的应用程序，需要向操作系统轮询文件符是否完成，导致应用程序忙于轮询，无暇使用cpu时间。

Node中的io操作不阻塞cpu计算：Node上调用io操作，会准备一个请求对象(包含请求/结果/回调)并新开一个线程，在这个线程上执行这个请求对象，io请求完成后线程把请求对象放入某容器。Node每次tick监控该容器，取出请求对象(事件)，并执行请求对象的回调(事件的回调)。

Node在window直接使用IOCP，在linux下利用epoll io自己实现了基于线程池的io操作，在libuv层完成兼容。IOCP：window提供的“线程池+阻塞IO”实现。

以给cpu输入文件为例，fs.open调libuv的uv_fs_open函数，uv_fs_open函数以FSReqWrap为参数调用IOCP的QueueUserWorkItem，fs.open不等待其执行完毕立即返回继续执行后续cpu计算，而IOCP的内部线程执行FSReqWrap，达到异步io的效果。io完成，IOCP内部线程通知IOCP(通过PostQueuedCompletionStatus)。Node每次tick都监控IOCP（通过GetQueuedCompletionStatus），将FSReqWrap作为事件放入自己的观察者队列，执行他的回调。

显示器即是io输入设备，又是io输出设备。chrome中主线程/tick线程大量计算会导致无法实时与io设备(显示器)进行io导致界面卡顿。Node中主线程/tick线程中如果有大量cpu计算会导致cpu无法得到输入、无法输出，导致io操作被cpu计算阻塞。所以，解决方案一是将计算外包出去，给其他线程，chrome的worker线程、Node的child_process。cluster模块基于child_process模块。解决方案二，建议一次tick中cpu计算不超过10ms，否则就使用setImmediate把cpu计算延后来保证请求io的实时性。

node中存在一些与io无关的异步操作，process.nextTick/setTimeout/setImmediate。每次tick，node对各类观察者进行回调执行，次序为：idle观察者/io观察者/check观察者。process.nextTick属于idle观察者。setImmediate是check观察者。

process.nextTick：nextTick维护一个数组，每次tick都会清空数组。

setTimeout：调用setTimeout，new一个定时器对象。定时器对象由定时器观察者管理，定时器观察者维护一颗红黑树。每次tick，取出一个超时的定时器形成事件执行回调。如果上一个tick用时过长，定时器回调就会超时执行。setTimeout(0)比process.nextTick慢，插入一个nextTick回调时间复杂度是O(1)，而插入一个setTimeout回调时间复杂度是O(logn)。

setImmediate：setImmediate维护一个链表，每次tick只取表头元素。所以，setImmediate的回调很可能不会在下次tick时执行、immediately执行。

可以如工厂一样批量创建函数的函数，称为偏函数

trycatch只捕获当前tick的异常

A观察B订阅B就是被B改变。监听发布者的事件(发布订阅模式)是一种广泛存在于node的编程模式，利用事件回调进行编程。因为事件名被所有观察者共享，所以，事件回调可以被所有观察者调用，而普通回调只能被父函数调用。比如要求同一时间同一sql不能重复请求，但同一时间许多请求者又需要sql结果，解决方案：让只有一个请求者emit selected，让所有请求者on selected，这样，一个请求者发出了请求，所有的请求者都得到了sql结果，利用的就是事件回调只认事件名所带来的全局性。

对于EventEmitter对象，如果监听了它的error事件，遇到报错时，是执行该事件回调，没有监听error事件才抛出异常。我们应该监听error事件，提高node线程健壮性。EventEmitter对象的观察者数量是被限制的，emitter.setMaxListener(0)可以解除该限制。

雪崩问题指当高访问高并发时期缓存失效，大量请求涌入数据库超过负载。启动程序时有高访问，此时缓存为空，很容易雪崩。

观察者可以选择只观察一次\只被改变一次，通过emitter.once('event1'，action)。

emit和on可以不一一对应，多个emit可以执行一个on，反之亦然。

事件回调的调用方式：拿到发布者的引用，通过事件字符串调用。

Promise所代表的那种模式在jQuery时代就被广为使用,Promise模式从2009年开始被抽象出来，陆续有A，B，D草案提交给CommonJS规范。
$.get('/api').success().success().error().complete(),把io操作封装成一个Promise对象。Promise的本质是一个发布者对象EventEmitter，Promise.resolve底层调用EventEmitter.emit。Promise.then底层EventEmitter.once。

```
const Promise = function(){
    EventEmitter.call(this)
}

```

Promise模式解决了串行io程序编写造成的回调地狱。Promise.all使得多发布单观察的程序编写变得简单。p88提供了一个Promise.all的实现，具体是，new Promise并维护一个计数count。为all中的每个Promise加一个观察者：count--，当count === 0时让promise发布all事件。

Promise.resolve时，将状态写为fulfilled并发布resolve事件。

Promise本质是一个发布者。而ajax本身也是一个发布者。★★Promise模式的本质是一个中间层，原本定义在ajax的回调，现在定义在Promise上。有了中间层就有了想象空间，就可以做一些事情，比如链式设置多个观察者\回调。具体实现是，当发布者发布resolve事件，会迭代调用观察者回调，如果返回的是新的Promise，则剩下未执行的观察者，转移为观察这个新Promise，即队列里剩下的回调转移给它。

在最初的Promise模式的Deferrred实现中，除了发布resolve/reject事件还可以发布progress事件。

如果对下层服务器发起大量并发io请求，下层服务器会吃不消。如果对文件系统发起大量io请求，文件描述符数量会被瞬间用光，解决方案：用队列做过载保护，定一个最大并发数。当超过最大并发值，不再发起io请求。当并发数降到该值以下，才发起io请求。针对队列被异常耗时的请求堵塞的情况，需要将超时的请求请出队列。可以在io请求发出的那一刻同时开启计时，计时先触发回调则意味着超时。针对没时间排队的io请求，提供拒绝排队机制，需要排队就直接报错返回。

### 第五章 内存控制

程序长时间不crash，就需要关注内存控制

编程 === 控制cpu和io设备

js的所有对象都生存在heap中。process.memoryUsage()

node初始化时，向操作系统申请一块内存，当申请的内存不够new新对象时再向操作系统申请。申请量达到某一极限值，node崩溃处理。人为设定极限值是因为node总内存大小与GC时间正相关。GC时js线程挂起，1.5G的堆内存，新对象GC一次需要50ms，老对象GC全停顿一次1s。所以node内存极限值约2G。

经历零或一次GC的对象 === 新对象。经历两次GC的对象 === 老对象。新对象和老对象放在不同的堆区域。node内存大小就是新对象空间大小和老对象空间大小的总和。新生代对象最大允许占32MB，老生代对象最大允许占1400MB，差了40倍。

新对象多为死亡对象，新对象GC使用的是Scavenge-Cheney算法，该算法发表在1970年的ACM上。原理是把新对象从原区域(semispace)复制转移到新区域，即From区域到To区域(新的From空间)。具体过程：遍历新对象，检查其是否存活，复制转移存活对象到To空间(或老对象空间)。因为新对象多为死亡对象，所以只有很少的存活对象被复制，所以这种只复制转移存活对象的算法让GC能完成得很快。

一个新对象会出生在From空间，经历一次GC，第二次经历GC时就会被复制转移到老对象空间。或者，如果GC复制的过程中，To空间已经被转移了4MB的新对象，则接下来From空间的剩余新对象一律直接转移为老对象，这是为了避免接下来新对象出生，空间不够。

老对象多为存活对象，所以使用Mark-Sweep/Mark-Compact这种每次只清理少数死亡对象的算法，让GC快速完成。Sweep算法，标记并清理未标记的死亡对象。Sweep的问题是，一次标记清理后，内存会出现不连续状态。这种内存状况，使空闲内存也无法被用来存放对象。

Compact算法解决了这个问题，Compact在Sweep算法的基础上，将存活对象往对象空间的一端复制转移，这样使得内存使用保持连续，缺点是Compact是三者中最慢的GC算法，只在对象空间不足以应对晋升过来的新对象时执行。

老对象GC全停顿一次1s，最简单的优化手段，是拆分执行/步进执行，将GC和js执行交替间隔执行，将每次的标记/清理/整理动作变为增量式的。

一个标签页 === v8实例。减少GC，是性能调优之一，应让GC尽可能少的发生。node --trace_gc查看GC日志。node --prof生成性能日志v8.log，v8.log使用linux-tick-processor打开。